In [1]:
import tensorflow as tf

/home/cyclops/anaconda3/envs/base_clone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../data/mnist/", one_hot=False)

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


## Build model

In [9]:
in_dim = mnist.train.images.shape[1]
encoder_dim = 200
out_dim = in_dim

In [42]:
pred_fn = tf.estimator.inputs.numpy_input_fn(x=mnist.test.images[:3], 
#                                               y=mnist.test.images,
                                              shuffle=False,
                                              num_epochs=None)

In [31]:
input_fn = tf.estimator.inputs.numpy_input_fn(x=mnist.train.images, 
                                              y=mnist.train.images,
                                              shuffle=True,
                                              num_epochs=None)

In [10]:
def net(x):
    encoder = tf.layers.dense(x, encoder_dim)
    out = tf.layers.dense(encoder, out_dim)
    return out

In [27]:
def model_fn(features, labels, mode):
    new_images = net(features)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=new_images) 
    
    loss_op = tf.reduce_mean(tf.pow(features - new_images, 2))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())    
    
    acc_op = tf.metrics.accuracy(labels=features, predictions=new_images)
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=new_images,
                                      loss=loss_op,
                                      train_op=train_op,
                                      eval_metric_ops={'accuracy': acc_op})

In [28]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpsesk2ulc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4ba01fed68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [37]:
model.train(input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpsesk2ulc/model.ckpt-1100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1101 into /tmp/tmpsesk2ulc/model.ckpt.
INFO:tensorflow:loss = 1050519.6, step = 1101
INFO:tensorflow:global_step/sec: 99.1096
INFO:tensorflow:loss = 551.4342, step = 1201 (1.010 sec)
INFO:tensorflow:global_step/sec: 116.528
INFO:tensorflow:loss = 1001.54834, step = 1301 (0.858 sec)
INFO:tensorflow:global_step/sec: 116.298
INFO:tensorflow:loss = 265.572, step = 1401 (0.860 sec)
INFO:tensorflow:global_step/sec: 119.704
INFO:tensorflow:loss = 24.653818, step = 1501 (0.835 sec)
INFO:tensorflow:global_step/sec: 117.7
INFO:tensorflow:loss = 2483.6106, step = 1601 (0.850 sec)
INFO:tensorflow:global_step/sec: 119.916
INFO:tensorflow:loss = 53.816257

In [43]:
list(model.predict(pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpsesk2ulc/model.ckpt-3100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


KeyboardInterrupt: 